In [3]:
import random
import copy
import matplotlib.pyplot as plt

class GeneticSearch:
    def __init__(self, params):
        self.repr_rate = params['repr_rate']
        self.num_agents = params['num_agents']
        
        self.max_iters = params['max_iters']
        self.stop_tol = params['stop_tol']
        self.stop_tol_iters = params['stop_tol_iters']
        
        self.constraints = params['constraints']
        
        self.data_collector = {
            'max_fitness': [],
            'avg_fitness': [],
            'num_mutations': [],
            'num_crossovers': []
        }

        self.Population = GeneticPopulation(params['p_mut'], params['p_cross'], params['gene_list'], params['fitness_function'])
    
    def eval_func(self, agent):
        
        return self.fitness_func(agent.gene)
    
    def step_vdps(self):
        self.repr_pairs = []
        self.reproduce(0)
        
        for pair in self.repr_pairs:
            pair[0].crossover(pair[1]) 
        
        for pair in self.repr_pairs:
            for agent in pair:
                agent.mutate()
        
        for pair in self.repr_pairs:
            for agent in pair:
                agent.fit = self.eval_func(agent)
    
    def step_hw(self):

        for i in range(len(self.agents)):
            for j in range(i+1, len(self.agents)):
                self.agents[i].crossover(self.agents[j])
                    
        
        for agent in self.agents:
            agent.hw_mutate()
            agent.fit = self.eval_func(agent)
            
        self.agents.append(self.Population.curr_elite)
        return 'Optimum: ' + str(best_agent.loc) + ' Fitness: ' + str(best_agent.fit)
    
                # Collect data
        self.data_collector['max_fitness'].append(self.Population.curr_elite.fit)
        self.data_collector['avg_fitness'].append(sum(self.Population.fits)/self.Population.num_agents)
            
            
    def step(self):
        
        # Create pairs
        # create new agents from crossover function
        repr_pairs = []
        new_agents = []
        
        for i in range(self.Population.num_agents//2):
            repr_pairs.append((self.Population.select_agent(), self.Population.select_agent()))
        
        new_agents = [pair[0].crossover(pair[1]) for pair in repr_pairs]
        new_agents = [agent for pair in new_agents for agent in pair]
        self.Population.agents = new_agents
        
        for agent in self.Population.agents:
            agent.hw_mutate()
        
        self.Population.agents.append(self.Population.curr_elite)
        self.Population.calc_fit_values()
        
#         for agent in self.Population.agents:
#             print(agent.gene)
    
    def run(self):
        
        diff = 0
        not_converged = True
        converged_count = 0
        self.iter_count = 0
        
        self.Population.init_agents()
        
        # Check that the dimensions of everything matches here
        
        # Begin running the genetic search
        while not_converged and (self.iter_count < self.max_iters):
            
            self.step_vdps()
            
            diff = self.Population.curr_elite.fit - self.Population.prev_elite.fit
            if abs(diff) < self.stop_tol:
                converged_count += 1
            else:
                converged_count = 0
                
            not_converged = True if converged_count < self.stop_tol_iters else False
            self.iter_count += 1
        
#         print(best)
            
        
    def reproduce(self, step_method):
        
        if step_method == 0:
            for i in range(self.repr_rate):
                agent1 = self.roulette_select(fs_roulette, random.random())
                agent2 = self.roulette_select(fs_roulette, random.random())

                self.repr_pairs.append((agent1, agent2))
                
        elif step_method == 1:
            new_agents = []
            for i in range(self.num_agents-1):
                ag = self.roulette_select(fs_roulette, random.random())
                new_agents.append(copy.deepcopy(ag))

            self.agents = new_agents
            return best_agent
        
    
    def display_2d(self):
        #Display current locations of agents
        plt.cla()
        plt.clf()
        x_list = [agent.loc[0] for agent in self.agents]
        y_list = [agent.loc[1] for agent in self.agents]
#         plt.scatter(x_list, y_list)
        plt.hexbin(x_list, y_list, gridsize=50, cmap='inferno')
        plt.show()

    
        

In [104]:
class GeneticPopulation:
    def __init__(self, p_mut, p_cross, gene_list, fitness_func):
#         self.p_mut = p_mut
#         self.p_cross = p_cross
        self.num_agents = 0
        self.agents = []
        self.fits = []
        self.curr_elite = None
        self.prev_elite = None
        self.roulette = []
        self.fitness_func = fitness_func
        
        # Initialize agents based on genome list
        self.agents = [GeneticAgent(gene, p_mut, p_cross) for gene in gene_list]
        self.num_agents = len(self.agents)
        self.curr_elite = random.choice(self.agents)
        self.calc_fit_values()
        self.calc_fit_values()
    
    def calc_fit_values(self):
        ''' Calculate '''
        
        # recalculate fitness for each agent here
        for agent in self.agents:
            agent.fit = self.fitness_func(agent.gene)
        
        self.prev_elite = self.curr_elite
        self.curr_elite = copy.deepcopy(min(self.agents, key=lambda x: x.fit))
        
        self.fits = [agent.fit for agent in self.agents]
        self.calc_roulette()

    
    def calc_roulette(self):
        ''' Normalize fits and create roulette wheel'''
        # TODO check logic here
#         fits_adj = [self.curr_elite.fit - fit for fit in self.fits]
#         fits_adj = [max(self.fits) - fit for fit in self.fits]
#         print('fits: ', self.fits)
        fits_inv = [1/fit for fit in self.fits]
        fits_inv_sum = sum(fits_inv)
#         print('fits_adj: ', fits_adj)
#         fits_norm = [fit/sum(fits_adj) for fit in fits_adj]
        fits_norm = [fit/fits_inv_sum for fit in fits_inv]
#         print('fits_norm: ', fits_norm)
        wheel = [0.0]
        
        for norm_fit in fits_norm:
            wheel.append(wheel[-1] + norm_fit)
        
        self.roulette = wheel
        
    def select_agent(self):
        ''' Select an agent proportional to fitness on roulette wheel'''
        
        r = random.random()
        for i in range(len(self.roulette)):
            if r < self.roulette[i]:
                return self.agents[i-1]

In [105]:
class GeneticAgent:
    def __init__(self, gene, p_mut, p_cross):
        self.gene = gene
        self.p_mut = p_mut
        self.p_cross = p_cross
    
    def crossover(self, other):
        
        gene1 = self.gene
        gene2 = other.gene
#         print('no crossover')
#         print('gene 1: ', gene1)
#         print('gene 2: ', gene2)
        if random.random() < self.p_cross:
#             print('crossover occured')
            left = random.randint(0, len(self.gene)-1)
            right = random.randint(0, len(self.gene)-1)     
            gene1[left:right+1], gene2[left:right+1] = other.gene[left:right+1], self.gene[left:right+1]
            
        return GeneticAgent(gene1, self.p_mut, self.p_cross), GeneticAgent(gene2, self.p_mut, self.p_cross)
    
    def mutate(self):
        if random.random() < self.p_mut:
            r = random.randint(0, len(self.gene)-1)
            self.gene[r] ^= 1
            
    def hw_mutate(self):
        for i in range(len(self.gene)):
            if random.random() < self.p_mut:
                self.gene[i] ^= 1

In [27]:
def rand_gene(n):
    binary = [0, 1]
    return [random.choice(binary) for i in range(n)]
    
    

# print(genomes)

# P = GeneticPopulation(.8, .7, genomes, lambda x: -sum(x))

### Confirm that roulette selection favors fitter agents

In [28]:
# print(P.fits)
# print(sum(P.fits)/len(P.fits))

# s = 0
# for i in range(1000):
#     s += P.select_agent().fit 

# print(s/1000)


In [29]:
g = GeneticAgent([0, 1, 1, 0, 1], 1, .4)

In [30]:
g.gene

[0, 1, 1, 0, 1]

In [31]:
g.hw_mutate()
g.gene

[1, 0, 0, 1, 0]

In [86]:
genomes = [rand_gene(15) for i in range(10)]

params = {
    'num_agents': 100,
    'fitness_function': lambda x: abs(5-sum(x))+1,
    'gene_list': genomes,
    'constraints': None,
    'p_mut': .03,
    'p_cross': .4,
    'repr_rate': 10,
    'max_iters': 300,
    'stop_tol': 10**(-8),
    'stop_tol_iters': 30
}

In [87]:
# G = GeneticSearch(params)
# G.Population.fits
# for agent in G.Population.agents:
#     print(agent.gene)

fits:  [6, 6, 4, 6, 4, 3, 4, 4, 3, 5]
fits_norm:  [0.0704225352112676, 0.0704225352112676, 0.1056338028169014, 0.0704225352112676, 0.1056338028169014, 0.1408450704225352, 0.1056338028169014, 0.1056338028169014, 0.1408450704225352, 0.08450704225352113]
fits:  [6, 6, 4, 6, 4, 3, 4, 4, 3, 5]
fits_norm:  [0.0704225352112676, 0.0704225352112676, 0.1056338028169014, 0.0704225352112676, 0.1056338028169014, 0.1408450704225352, 0.1056338028169014, 0.1056338028169014, 0.1408450704225352, 0.08450704225352113]
[1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
[1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0]
[1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]
[0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0]


In [88]:
# G.step()
    
# for agent in G.Population.agents:
#     print(agent.gene)

no crossover
gene 1:  [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
gene 2:  [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0]
no crossover
gene 1:  [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1]
gene 2:  [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
no crossover
gene 1:  [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]
gene 2:  [1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1]
crossover occured
no crossover
gene 1:  [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0]
gene 2:  [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
no crossover
gene 1:  [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
gene 2:  [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
fits:  [4, 6, 4, 4, 6, 3, 6, 3, 5, 4, 3]
fits_norm:  [0.09259259259259259, 0.061728395061728385, 0.09259259259259259, 0.09259259259259259, 0.061728395061728385, 0.12345679012345677, 0.061728395061728385, 0.12345679012345677, 0.07407407407407407, 0.09259259259259259, 0.12345679012345677]
[1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,